# CMPE 484: Bioinformatics and Computational Genomics, Spring 2022
## Assignment I -  Pairwise Sequence Alignment

Please describe your work clearly since this notebook is considered as your report.

# ADALET VEYİS TURGUT 2017400210

## helpers

In [3]:
!pip3 install blosum 
import blosum as bl
blosumDict = {key:int(val) for key,val in dict(bl.BLOSUM(62)).items()}
#usage: blosumDict['GC']

Defaulting to user installation because normal site-packages is not writeable


In [4]:
def zeros(hor,ver):
    """
    create a matrix full of zeros with given sizes
    """
    matrix = []
    for i in range(hor):
        matrix.append([])
        for j in range(ver):
            matrix[-1].append(0)
    return matrix
def print_matrix(matrix):
    """
    print matrix with with tabs between elements
    """
    for row in matrix:
        for el in row:
            print(el, end="\t")
        print()
def find_index(matrix, elem):
    for i,row in enumerate(matrix):
        for j,el in enumerate(row):
            if el == elem:
                return i,j
    return -1,-1

## Score calculations

### linear score

In [15]:
def linear_score(seq1, seq2, penalty, is_local=False):
    """
    calculate alignment scores with linear penalty.
    to decide a score, we have three options: we can come from above, from left or from upper left corner.
    if we come from above or left, we add penalty; if we come from the upper left corner, we add 
        match-mismatch value (found from blosum matrix) to the position we came from.
    if algorithm is local, we also consider 0.
    we take maximum of them
    we do the steps above for every entry in the matrix.
    """
    matrix = zeros(len(seq1)+1,len(seq2)+1)
    traceback_matrix = zeros(len(seq1)+1,len(seq2)+1)
    label_dict = {0:'corner', 1:'left', 2:'up', 3:'switch'}
    if not is_local:
        for idx,_ in enumerate(matrix[0]):
            matrix[0][idx] = idx * penalty
        for idx,_ in enumerate(matrix):
            matrix[idx][0] = idx * penalty
    for i in range(1,len(matrix)):
        for j in range(1,len(matrix[i])):
            blosum_score = 1 if seq1[i-1] == seq2[j-1] else -1
            #entries = [matrix[i-1][j-1] + blosumDict["%c%c" % (seq1[i-1],seq2[j-1])],matrix[i][j-1] + penalty, matrix[i-1][j] + penalty]
            entries = [matrix[i-1][j-1] + blosum_score,matrix[i][j-1] + penalty, matrix[i-1][j] + penalty]
            if is_local:
                entries.append(0)
            matrix[i][j] = max(entries)
            if matrix[i][j] == 0: traceback_matrix[i][j] = 'switch'
            else: traceback_matrix[i][j] = label_dict[entries.index(matrix[i][j])]
    return matrix, traceback_matrix

### affine score

In [6]:
def affine_score(seq1, seq2, op_penalty, ext_penalty, is_local=False):
    """
    calculate alignment scores with affine penalty.
    now we have three different matrices (or you can say layers): upper matrix for vertical edges: layer_matrixes[2], lower matrix
        for horizantal edges: layer_matrixes[1] and main matrix for diagonal edges: layer_matrixes[0]
    we calculate scores for each of the matrices: if we come from another matrix, we add open penalty; if we come from same matrix, 
        we add extension penalty. If is_local is true, we also consider 0 which means we fresh start a new partial alignment.
    we do the steps above for every entry in the matrices.
    final score is the maximum element of these matrices.
    we also create traceback_matrices for each layer to keep track of where we came from. 
    """
    # initialize layer matrices
    layer_matrixes = [zeros(len(seq1)+1,len(seq2)+1) for _ in range(3)] # [main_level, lower_level, upper_level]
    layer_matrixes[0][0][0] = 0
    layer_matrixes[1][0][0] = op_penalty
    layer_matrixes[2][0][0] = op_penalty
    for i in range(1,len(seq1)+1):
        layer_matrixes[0][i][0] = -10000
        layer_matrixes[2][i][0] = -10000
        layer_matrixes[1][i][0] = op_penalty + ext_penalty * (i-1)
    layer_matrixes[0][0][1:] = [-10000 for i in range(len(seq2))]
    layer_matrixes[1][0][1:] = [-10000 for i in range(len(seq2))]
    layer_matrixes[2][0][1:] = [op_penalty + ext_penalty * i for i in range(len(seq2))]
    # initialize traceback matrices
    traceback_matrices = [zeros(len(seq1)+1,len(seq2)+1) for _ in range(3)]
    # helper dict fill traceback matrix
    label_dict = {0:'main', 1:'lower', 2:'upper',3:'0',4:'blosum'}# lower:horizantal, upper:vertical
    # algorithm
    for i in range(1,len(layer_matrixes[0])):
        for j in range(1,len(layer_matrixes[0][i])):
            blosum_score = blosumDict["%c%c" % (seq1[i-1],seq2[j-1])]
            if is_local:
                entries = [layer_matrixes[k][i-1][j-1] + blosum_score for k in range(3)]+[0, blosum_score]
            else:
                entries = [layer_matrixes[k][i-1][j-1] + blosum_score for k in range(3)]
            layer_matrixes[0][i][j] = max(entries)
            traceback_matrices[0][i][j] = label_dict[entries.index(layer_matrixes[0][i][j])]
            layer_matrixes[1][i][j] = max(layer_matrixes[1][i][j-1] + ext_penalty, layer_matrixes[0][i][j-1] + op_penalty)
            traceback_matrices[1][i][j] = 'lower' if layer_matrixes[1][i][j]==layer_matrixes[1][i][j-1] + ext_penalty else 'main'
            layer_matrixes[2][i][j] = max(layer_matrixes[2][i-1][j] + ext_penalty, layer_matrixes[0][i-1][j] + op_penalty )
            traceback_matrices[2][i][j] = 'upper' if layer_matrixes[2][i][j]==layer_matrixes[2][i-1][j] + ext_penalty else 'main'
            entries = [layer_matrixes[k][i][j] for k in range(3)]
            #trace[i][j] = label_dict[entries.index(max(entries))]
    return layer_matrixes, traceback_matrices

## Backtrack functions

### linear backtrack

In [7]:
def linear_backtrack(seq1, seq2, trace_matrix, matrix=None, is_local=False):
    """
    we have a helper matrix, we follow what the matrix says to track back, we equalize the lengths of the sequences before 
    returning to make sure they are visually aligned.
    """
    if is_local:
        i,j = find_index(matrix,max(map(max,matrix)))
        new_seq1 = seq1[i:]
        new_seq2 = seq2[j:]
        mid_seq = ""
        ### append blank chars to shorter strings so that their lengths are aligned
        max_len = max(len(seq1[i:]),len(seq2[j:]))
        if len(new_seq1[i:]) < max_len:
            new_seq1 += ' '* (max_len-len(seq1[i:]))
        if len(seq2[j:]) < max_len:
            new_seq2 += ' '* (max_len-len(seq2[j:])) 
        if len(mid_seq) < max_len:
            mid_seq += ' '* (max_len-len(mid_seq)) 
    else:
        new_seq1 = ""
        new_seq2 = ""
        mid_seq = ""
        i = len(seq1)
        j = len(seq2)
    while i > 0 or j > 0:
        if trace_matrix[i][j] == 'corner':
            new_seq1 = seq1[i-1] + new_seq1
            new_seq2 = seq2[j-1] + new_seq2
            i -= 1
            j -= 1
            mid_seq = ('|' if seq1[i]==seq2[j] else '.') + mid_seq
        elif trace_matrix[i][j] == 'up':
            new_seq1 = seq1[i-1] + new_seq1
            new_seq2 = '-' + new_seq2
            i -= 1
            mid_seq = '-' + mid_seq
        elif trace_matrix[i][j] == 'left':
            new_seq1 = '-' + new_seq1
            new_seq2 = seq2[j-1] + new_seq2
            j -= 1
            mid_seq = '-' + mid_seq
        else: break
    new_seq1 = seq1[:i] + new_seq1
    new_seq2 = seq2[:j] + new_seq2
    max_len = max(len(new_seq1),len(new_seq2))
    fill_blank_char = ' ' if is_local else '-'

    if len(new_seq1) < max_len:
        new_seq1 = fill_blank_char* (max_len-len(new_seq1)) + new_seq1
    if len(new_seq2) < max_len:
        new_seq2 = fill_blank_char* (max_len-len(new_seq2)) + new_seq2
    if len(mid_seq) < max_len:
        mid_seq = fill_blank_char* (max_len-len(mid_seq)) + mid_seq
    return new_seq1, mid_seq, new_seq2

### affine backtrack

In [8]:
def affine_backtrack(seq1,seq2,layer_matrices,trace_matrices, is_local=False):
    """
    we have three helper trace matrices, one for each layer.
    First we find start position: for local, we find the index of the maximum element of these matrices; for global, we find maxium of the last entries. 
    we have two helper variables: prev_matrix_index and prev_matrix_name. They help us move back and forth between matrices and backtrack accordingly.
    After completing while loop, we fill the remaining spaces.
    """
    if is_local:
        max_of_three =  [max(map(max,matrix)) for matrix in layer_matrices]
        prev_matrix_index = max_of_three.index(max(max_of_three))
        i,j = find_index(layer_matrices[prev_matrix_index],max(max_of_three))
        new_seq1 = seq1[i:]
        new_seq2 = seq2[j:]
        mid_seq = ""
        ### append blank chars to shorter strings so that their lengths are aligned
        max_len = max(len(seq1[i:]),len(seq2[j:]))
        if len(new_seq1[i:]) < max_len:
            new_seq1 += ' '* (max_len-len(seq1[i:]))
        if len(seq2[j:]) < max_len:
            new_seq2 += ' '* (max_len-len(seq2[j:])) 
        if len(mid_seq) < max_len:
            mid_seq += ' '* (max_len-len(mid_seq))
    else:
        last_elements = [matrix[-1][-1] for matrix in layer_matrices]
        prev_matrix_index = last_elements.index(max(last_elements))
        i,j = len(seq1),len(seq2)
        new_seq1,new_seq2,mid_seq = "","",""
    while i > 0 and j > 0:
        if prev_matrix_index == 0:
            prev_matrix_name = trace_matrices[0][i][j]
            if str(prev_matrix_name) == '0':break
            new_seq1 = seq1[i-1] + new_seq1
            new_seq2 = seq2[j-1] + new_seq2
            mid_seq = ('|' if seq1[i-1]==seq2[j-1] else '.') + mid_seq
            i -= 1
            j -= 1
            if prev_matrix_name == 'upper':
                prev_matrix_index = 2
            elif prev_matrix_name == 'lower':
                prev_matrix_index = 1
        elif prev_matrix_index == 2:
            prev_matrix_name = trace_matrices[2][i][j]
            if str(prev_matrix_name) == '0':break
            new_seq1 = seq1[i-1] + new_seq1
            new_seq2 = '-' + new_seq2
            mid_seq = '-' + mid_seq
            i -= 1
            if prev_matrix_name == 'main':
                prev_matrix_index = 0
            elif str(prev_matrix_name) == '0': break
        elif prev_matrix_index == 1:
            prev_matrix_name = trace_matrices[1][i][j]
            if str(prev_matrix_name) == '0':break
            new_seq1 = '-' + new_seq1
            new_seq2 = seq2[j-1] + new_seq2
            mid_seq = '-' + mid_seq
            j -= 1
            if prev_matrix_name == 'main':
                prev_matrix_index = 0
            elif str(prev_matrix_name) == '0': break
    new_seq1 = seq1[:i] + new_seq1
    new_seq2 = seq2[:j] + new_seq2
    max_len = max(len(new_seq1),len(new_seq2))
    fill_blank_char = ' ' if is_local else '-'
    if len(new_seq1) < max_len:
        new_seq1 = fill_blank_char* (max_len-len(new_seq1)) + new_seq1
    if len(new_seq2) < max_len:
        new_seq2 = fill_blank_char* (max_len-len(new_seq2)) + new_seq2
    if len(mid_seq) < max_len:
        mid_seq = fill_blank_char * (max_len-len(mid_seq)) + mid_seq
    return new_seq1, mid_seq, new_seq2

# Smith Waterman (Local Sequence Alignment)

In [21]:
def smith_waterman_algorithm(seq1, seq2, penalty_params):
    if penalty_params['penalty_type'] == 'linear':
        matrix, trace_matrix = linear_score(seq1, seq2, penalty_params['gap_opening_penalty'], True)
        print_matrix(matrix)
        print("Score:" ,max(map(max,matrix)))
        print(*linear_backtrack(seq1, seq2, trace_matrix, matrix ,True),sep="\n")

    elif penalty_params['penalty_type'] == 'affine': 
        layer_matrices, trace_matrices = affine_score(seq1, seq2, penalty_params['gap_opening_penalty'],penalty_params['gap_extension_penalty'], True)
        print("Score:" ,max([max(map(max,matrix)) for matrix in layer_matrices]))
        print(*affine_backtrack(seq1,seq2,layer_matrices,trace_matrices, True),sep="\n")
    else:
        raise ValueError("penalty_type must be linear or affine.")

# Needleman Wunsch (Global Sequence Alignment)

In [13]:
def needleman_wunsch_algorithm(seq1, seq2, penalty_params):
    if penalty_params['penalty_type'] == 'linear':
        matrix, trace_matrix = linear_score(seq1, seq2, penalty_params['gap_opening_penalty'])
        print("Score:" , matrix[-1][-1])
        print_matrix(matrix)
        print(*linear_backtrack(seq1,seq2,trace_matrix),sep="\n")
    elif penalty_params['penalty_type'] == 'affine':
        layer_matrices, trace_matrices = affine_score(seq1, seq2, penalty_params['gap_opening_penalty'],penalty_params['gap_extension_penalty'])
        print("Score:",max([layer_matrices[k][-1][-1] for k in range(3)]))
        print(*affine_backtrack(seq1,seq2,layer_matrices,trace_matrices),sep="\n")
    else:
        raise ValueError("penalty_type must be linear or affine.")

# Generic Sequence Alignment Function

In [11]:
def align_sequences(seq1, seq2, algorithm, penalty_params):
    if len(seq1)==0 or len(seq2) == 0:
        raise ValueError("Sequence(s) cannot be 0 length!")
    if algorithm == 'global':
        return needleman_wunsch_algorithm(seq1, seq2, penalty_params)
    elif algorithm == 'local': 
        return smith_waterman_algorithm(seq1, seq2, penalty_params)
    else:
        raise NotImplementedError

In [19]:
align_sequences(sample1_sequence2, sample1_sequence1, 'global', {'penalty_type': 'linear', 'gap_opening_penalty': -1}) # works


Score: -1
0	-1	-2	-3	-4	-5	-6	-7	-8	-9	
-1	-1	-2	-3	-2	-3	-4	-5	-6	-7	
-2	-2	-2	-3	-2	-1	-2	-3	-4	-5	
-3	-3	-1	-2	-3	-2	-2	-3	-2	-3	
-4	-4	-2	0	-1	-2	-3	-3	-3	-3	
-5	-5	-3	-1	1	0	-1	-2	-3	-4	
-6	-4	-4	-2	0	0	-1	0	-1	-2	
-7	-5	-3	-3	-1	-1	-1	-1	1	0	
-8	-6	-4	-2	-2	-2	-2	-2	0	0	
-9	-7	-5	-3	-1	-1	-1	-2	-1	-1	
GGACGTACG
--------.
TACGGGTAT


In [23]:
align_sequences(sample1_sequence2, sample1_sequence1, 'local', {'penalty_type': 'linear', 'gap_opening_penalty': -1})

0	0	0	0	0	0	0	0	0	0	
0	0	0	0	1	1	1	0	0	0	
0	0	0	0	1	2	2	1	0	0	
0	0	1	0	0	1	1	1	2	1	
0	0	0	2	1	0	0	0	1	1	
0	0	0	1	3	2	1	0	0	0	
0	1	0	0	2	2	1	2	1	1	
0	0	2	1	1	1	1	1	3	2	
0	0	1	3	2	1	0	0	2	2	
0	0	0	2	4	3	2	1	1	1	
Score: 4
GGACGTACG     
     ||||     
     TACGGGTAT


# Example calls

In [10]:
sample1_sequence1 = 'AEPWAHT'
sample1_sequence2 = 'CDKIP'
align_sequences(sample1_sequence1, sample1_sequence2, 'local', {'penalty_type': 'linear', 'gap_opening_penalty': -1})
print()
align_sequences(sample1_sequence1, sample1_sequence2, 'local', {'penalty_type': 'affine','gap_opening_penalty': -5, 'gap_extension_penalty': -2 })
print()
align_sequences(sample1_sequence1, sample1_sequence2, 'global', {'penalty_type': 'linear', 'gap_opening_penalty': -1}) # works
print()
align_sequences(sample1_sequence1, sample1_sequence2, 'global', {'penalty_type': 'affine', 'gap_opening_penalty': -11, 'gap_extension_penalty': -1})

Score: 7
  AEPWAHT
    |    
CDKIP    

Score: 7
  AEPWAHT
    |    
CDKIP    

Score: 3
--AEPWAHT
----|----
CDKIP----

Score: -15
AEPWAHT
..--...
CD--KIP


# Given code

## Inputs

You can use the functions below to generate sample sequences to test your algorithms. 

In [11]:
import random
aminoacids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

def generate_sequence(length=50):    
    return ''.join([random.choice(aminoacids) for i in range(length)])

def mutate_sequence(seq, n_mutations=10):
    seq = list(seq)
    pos = {random.randint(1, len(seq)): random.choice(['substitute', 'delete']) for i in range(n_mutations)}   
    mutated_sequence = ''
    for ix, aminoacid in enumerate(seq):
        if ix in pos:
            if pos[ix] == 'substitute':
                mutated_sequence += random.choice(aminoacids)
        else:
            mutated_sequence += aminoacid             
    return mutated_sequence

sample_sequence = generate_sequence()
sequence1 = mutate_sequence(sample_sequence)
sequence2 = mutate_sequence(sample_sequence)
sequence1, sequence2

('LGAAWYTGCCFIGNHEIAHRCQEVHIGSRAQHCVTPTHDAHWERY',
 'LGMAWVTGCCFIWLGNHEVAHRLFEHIIGDRAQDRVTPTHDAHVEYRP')

#### Sample 1

In [17]:
sample1_sequence1 = 'TACGGGTAT'
sample1_sequence2 = 'GGACGTACG'

In [1]:
!pip3 install bio
from Bio.Align import substitution_matrices
from Bio import Align 
def align_sequences_biopython(seq1, seq2, algorithm, penalty_params):
    aligner = Align.PairwiseAligner()
    aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
    if penalty_params['penalty_type'] == 'linear':
        aligner.open_gap_score = penalty_params['gap_opening_penalty'] 
        aligner.extend_gap_score = penalty_params['gap_opening_penalty'] 
    else:
        aligner.open_gap_score = penalty_params['gap_opening_penalty'] # -11
        aligner.extend_gap_score = penalty_params['gap_extension_penalty'] # -1
    aligner.mode = algorithm
    for alignment in aligner.align(seq1, seq2):
        print('Score:', alignment.score)
        print(alignment)


Defaulting to user installation because normal site-packages is not writeable


In [14]:
align_sequences_biopython(sample1_sequence1, sample1_sequence2, 'local', {'penalty_type': 'linear', 'gap_opening_penalty': -1})

Score: 14.0
AATVC--GSATK-YA
    |--|---.-.
    CTWG---RCF



In [15]:
align_sequences_biopython(sample1_sequence1, sample1_sequence2, 'local', {'penalty_type': 'affine', 'gap_opening_penalty': -5, 'gap_extension_penalty': -2})

Score: 9.0
AATVCGSATKYA
    |
    CTWGRCF

Score: 9.0
 AATVCGSATKYA
     |
CTWGRCF



In [16]:
align_sequences_biopython(sample1_sequence1, sample1_sequence2, 'global', {'penalty_type': 'linear', 'gap_opening_penalty': -1})

Score: 9.0
AATVC--GSATK-YA
----|--|---.-.-
----CTWG---RCF-



In [17]:
align_sequences_biopython(sample1_sequence1, sample1_sequence2, 'global', {'penalty_type': 'affine', 'gap_opening_penalty': -11, 'gap_extension_penalty': -1})

Score: -21.0
AATVCGSATKYA
.-----......
C-----TWGRCF

Score: -21.0
AATVCGSATKYA
...-----....
CTW-----GRCF



#### Sample 2

In [18]:
sample2_sequence1 = 'PRTEINS'
sample2_sequence2 = 'PRTWPSEIN'

In [19]:
align_sequences_biopython(sample2_sequence1, sample2_sequence2, 'local', {'penalty_type': 'linear', 'gap_opening_penalty': -1})

Score: 29.0
PRT---EINS
|||---|||
PRTWPSEIN



#### Sample 3

In [20]:
sample1_sequence1 = 'WAHYHFDVPDCWAHRYWVENPQAIAQMEQICFWIPAVLSIDHEQMNWFPSMMMKQPHVFKVDHHMSCRWLPIRGKKCSSCCTRMRVRTVWE'
sample1_sequence2 = 'YHEDVAHEDAIAQMVNTFGFVWQICLNQFPSMWIPMMKIYWIAVLSAHVADRKTWSKHMSCRWLPIISATCARMRVRTVWE'